In [1]:
import pygame, sys, random
from pygame.math import Vector2

class Food:
    def __init__(self, snake_body):
     self.position = self.generate_random_pos(snake_body)
    
    def draw(self):
        food_rect = pygame.Rect(OFFSET + self.position.x*cell_size, OFFSET + self.position.y*cell_size, cell_size, cell_size)
        screen.blit(food_surface, food_rect)
        #pygame.draw.rect(screen, DARK_GREEN, food_rect)
    
    def generate_random_cell(self):
        x = random.randint(0, no_of_cells -1)
        y = random.randint(0, no_of_cells -1)
        position = Vector2(x, y)
        return position
    
    def generate_random_pos(self, snake_body):
        position = self.generate_random_cell()
        while position in snake_body:
            position = self.generate_random_cell()
        return position
    
class Snake:
	def __init__(self):
		self.body = [Vector2(5,10),Vector2(4,10),Vector2(3,10)]
		self.direction = Vector2(0,0)
		self.new_block = False

		self.head_up = pygame.image.load('head_up.png').convert_alpha()
		self.head_down = pygame.image.load('head_down.png').convert_alpha()
		self.head_right = pygame.image.load('head_right.png').convert_alpha()
		self.head_left = pygame.image.load('head_left.png').convert_alpha()
		
		self.tail_up = pygame.image.load('tail_up.png').convert_alpha()
		self.tail_down = pygame.image.load('tail_down.png').convert_alpha()
		self.tail_right = pygame.image.load('tail_right.png').convert_alpha()
		self.tail_left = pygame.image.load('tail_left.png').convert_alpha()

		self.body_vertical = pygame.image.load('body_vertical.png').convert_alpha()
		self.body_horizontal = pygame.image.load('body_horizontal.png').convert_alpha()

		self.body_tr = pygame.image.load('body_tr.png').convert_alpha()
		self.body_tl = pygame.image.load('body_tl.png').convert_alpha()
		self.body_br = pygame.image.load('body_br.png').convert_alpha()
		self.body_bl = pygame.image.load('body_bl.png').convert_alpha()
		self.crunch_sound = pygame.mixer.Sound('bg_music_1.mp3')

	def draw(self):
		self.update_head_graphics()
		self.update_tail_graphics()

		for index,block in enumerate(self.body):
			x_pos = int(block.x * cell_size)
			y_pos = int(block.y * cell_size)
			block_rect = pygame.Rect(x_pos,y_pos,cell_size,cell_size)

			if index == 0:
				screen.blit(self.head,block_rect)
			elif index == len(self.body) - 1:
				screen.blit(self.tail,block_rect)
			else:
				previous_block = self.body[index + 1] - block
				next_block = self.body[index - 1] - block
				if previous_block.x == next_block.x:
					screen.blit(self.body_vertical,block_rect)
				elif previous_block.y == next_block.y:
					screen.blit(self.body_horizontal,block_rect)
				else:
					if previous_block.x == -1 and next_block.y == -1 or previous_block.y == -1 and next_block.x == -1:
						screen.blit(self.body_tl,block_rect)
					elif previous_block.x == -1 and next_block.y == 1 or previous_block.y == 1 and next_block.x == -1:
						screen.blit(self.body_bl,block_rect)
					elif previous_block.x == 1 and next_block.y == -1 or previous_block.y == -1 and next_block.x == 1:
						screen.blit(self.body_tr,block_rect)
					elif previous_block.x == 1 and next_block.y == 1 or previous_block.y == 1 and next_block.x == 1:
						screen.blit(self.body_br,block_rect)

	def update_head_graphics(self):
		head_relation = self.body[1] - self.body[0]
		if head_relation == Vector2(1,0): self.head = self.head_left
		elif head_relation == Vector2(-1,0): self.head = self.head_right
		elif head_relation == Vector2(0,1): self.head = self.head_up
		elif head_relation == Vector2(0,-1): self.head = self.head_down

	def update_tail_graphics(self):
		tail_relation = self.body[-2] - self.body[-1]
		if tail_relation == Vector2(1,0): self.tail = self.tail_left
		elif tail_relation == Vector2(-1,0): self.tail = self.tail_right
		elif tail_relation == Vector2(0,1): self.tail = self.tail_up
		elif tail_relation == Vector2(0,-1): self.tail = self.tail_down

	def move_snake(self):
		if self.new_block == True:
			body_copy = self.body[:]
			body_copy.insert(0,body_copy[0] + self.direction)
			self.body = body_copy[:]
			self.new_block = False
		else:
			body_copy = self.body[:-1]
			body_copy.insert(0,body_copy[0] + self.direction)
			self.body = body_copy[:]

	def add_block(self):
		self.new_block = True

	def play_crunch_sound(self):
		self.crunch_sound.play()

	def reset(self):
		self.body = [Vector2(5,10),Vector2(4,10),Vector2(3,10)]
		self.direction = Vector2(0,0)


class Game:
    def __init__(self):
        self.play_background_music()
        self.snake = Snake()
        self.food = Food(self.snake.body)
        self.state = "RUNNING"
        self.score = 0

    def play_background_music(self):
        pygame.mixer.music.load("bg_music_1.mp3")
        pygame.mixer.music.play()
        
    def draw(self):
        self.food.draw()
        self.snake.draw()
        
    def update(self):
        if self.state == "RUNNING":
            self.snake.move_snake()
            self.check_collision_with_food()
            self.check_collision_with_edges()
            self.check_collision_with_tail()
        
        
    def check_collision_with_food(self):
        if self.snake.body[0] == self.food.position:
            print("Eating Food")
            self.food.position = self.food.generate_random_pos(self.snake.body)
            self.snake.add_segment = True
            self.score += 1
            self.snake.eat_sound.play()
    
    def check_collision_with_edges(self):
        if self.snake.body[0].x == no_of_cells or self.snake.body[0].x == -1:
            self.game_over()
        if self.snake.body[0].y == no_of_cells or self.snake.body[0].y == -1:
            self.game_over()

    def check_collision_with_tail(self):
        headless_body = self.snake.body[1:]
        if self.snake.body[0] in headless_body:
            self.game_over()
            
        
            
    def game_over(self):
        print("Game Over")
        #raise "Game Over"
        self.snake.reset()
        self.food.position = self.food.generate_random_pos(self.snake.body)
        self.state = "STOPPED"
        self.score = 0
        self.snake.wall_hit_sound.play()
        #self.show_game_over()

pygame.init()

title_font = pygame.font.Font(None, 30)
score_font = pygame.font.Font(None, 40)

GREEN = (173,204,96)
DEEP_TEAL = (23, 77, 78)
DEEP_YELLOW = (255, 204, 0)
BROWN = (139, 69, 19)
RED = (255, 0, 0)
SOFT_PURPLE = (204, 153, 255)
SOFT_BLUE = (173, 216, 230)

cell_size = 25
no_of_cells = 18

OFFSET = 35 

screen = pygame.display.set_mode((2 * OFFSET + cell_size*no_of_cells, 2 * OFFSET + cell_size*no_of_cells))
pygame.display.set_caption("Vintage Viper")
clock = pygame.time.Clock()
#food = Food()
#snake = Snake()
game = Game()

food_surface = pygame.image.load("ratta.png")

SNAKE_UPDATE = pygame.USEREVENT
pygame.time.set_timer(SNAKE_UPDATE, 200)

while True:
    for event in pygame.event.get():
        if event.type == SNAKE_UPDATE:
            game.update()
            
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
            
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and game.snake.direction != Vector2(0, 1):
                game.snake.direction = Vector2(0, -1)
            if event.key == pygame.K_DOWN and game.snake.direction != Vector2(0, -1):
                game.snake.direction = Vector2(0, 1)
            if event.key == pygame.K_LEFT and game.snake.direction != Vector2(1, 0):
                game.snake.direction = Vector2(-1, 0)
            if event.key == pygame.K_RIGHT and game.snake.direction != Vector2(-1, 0):
                game.snake.direction = Vector2(1, 0)
                
        
    #drawing        
    screen.fill(GREEN)
    pygame.draw.rect(screen, SOFT_BLUE, (OFFSET-5, OFFSET-5, cell_size*no_of_cells+10, cell_size*no_of_cells+10), 5)
    game.draw()
    #food.draw()
    #snake.draw()
    
    title_surface = title_font.render("Vintage Viper", True, RED)
    score_surface = score_font.render(str(game.score), True, RED)
    screen.blit(title_surface,( OFFSET-5, 10))
    screen.blit(score_surface,( OFFSET-5, OFFSET + cell_size*no_of_cells+10))
    
    pygame.display.update()
    clock.tick(60)

pygame 2.5.2 (SDL 2.28.3, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Game Over


AttributeError: 'Snake' object has no attribute 'wall_hit_sound'